In [74]:
import pickle 
import pandas as pd

dbfile = open('dico_coef', 'rb')    
dico_coef_global = pickle.load(dbfile)

dbfile = open('dico_model_coefs', 'rb')    
dico_model_coefs = pickle.load(dbfile)

data = pd.read_pickle('data.pkl')

In [7]:
import gurobipy as gp
from gurobipy import GRB

In [75]:
var_dec = list(dico_coef_global.keys())
var_dec.remove('epentr_1')
var_dec.remove('larg')
var_dec.remove('intercept')
var_dec.remove('esort_7')




In [76]:
data = data[data.index>="31/12/2024 10:21:49"]
date_id = "31/12/2024 16:55:44"
ep_entree = data.loc[date_id,'epentr_1']
larg      = data.loc[date_id,'larg']
esort     = data.loc[date_id,'esort_7']

## Variable de décision

In [102]:
m = gp.Model('laminage')
x = {}

for v in var_dec:
    x[v] = m.addVar(name = v)

m.update()

## Fonction objectif

In [103]:
m.setObjective(gp.quicksum(dico_coef_global[e]*x[e] for e in var_dec)+
               dico_coef_global['epentr_1']*ep_entree+
               dico_coef_global['larg']*larg+
               dico_coef_global['esort_7']*esort+
               dico_coef_global['intercept'], GRB.MINIMIZE)

## Contraintes

In [104]:
epsilon = 0.00000001

esort1_ct = m.addConstr(x['esort_1'] <= ep_entree*(1.0-0.25), name='esort1_ct')
esort2_ct = m.addConstr(x['esort_2']<=x['esort_1']*(1.0-0.25), name='esort2_ct')
esort3_ct = m.addConstr(x['esort_3']<=x['esort_2']*(1.0-0.25), name='esort3_ct')
esort4_ct = m.addConstr(x['esort_4']<=x['esort_3']*(1.0-0.25), name='esort4_ct')
esort5_ct = m.addConstr(x['esort_5']<=x['esort_4']*(1.0-0.25), name='esort5_ct')
esort6_ct = m.addConstr(x['esort_6']<=x['esort_5']*(1.0-0.25), name='esort6_ct')

esort6_ct2 = m.addConstr(x['esort_6']>=esort+epsilon, name='esort6_ct2')



### Contraintes sur les puissance maximales des equipements

In [105]:
pmax = dict()

pmax['f1'] = 10.5
pmax['f2'] = 12.0
pmax['f3'] = 12.5
pmax['f4'] = 11.0
pmax['f5'] = 12.9
pmax['f6'] = 10.6
pmax['f7'] = 5.5



pmax_ct_1 = m.addConstr(dico_model_coefs['model_f1']['larg']*larg + 
                        dico_model_coefs['model_f1']['epentr_1']*ep_entree + 
                        dico_model_coefs['model_f1']['esort_1']*x['esort_1'] + 
                        dico_model_coefs['model_f1']['intercept'] <= pmax['f1'], name='pmax_f1')


pmax_ct_2 = m.addConstr(dico_model_coefs['model_f2']['larg']*larg + 
                        dico_model_coefs['model_f2']['esort_1']*x['esort_1'] + 
                        dico_model_coefs['model_f2']['esort_2']*x['esort_2'] + 
                        dico_model_coefs['model_f2']['intercept'] <= pmax['f2'], name='pmax_f2')

pmax_ct_3 = m.addConstr(dico_model_coefs['model_f3']['larg']*larg + 
                        dico_model_coefs['model_f3']['esort_2']*x['esort_2'] + 
                        dico_model_coefs['model_f3']['esort_3']*x['esort_3'] + 
                        dico_model_coefs['model_f3']['intercept'] <= pmax['f3'], name='pmax_f3')


pmax_ct_4 = m.addConstr(dico_model_coefs['model_f4']['larg']*larg + 
                        dico_model_coefs['model_f4']['esort_3']*x['esort_3'] + 
                        dico_model_coefs['model_f4']['esort_4']*x['esort_4'] + 
                        dico_model_coefs['model_f4']['intercept'] <= pmax['f4'], name='pmax_f4')


pmax_ct_5 = m.addConstr(dico_model_coefs['model_f5']['larg']*larg + 
                        dico_model_coefs['model_f5']['esort_5']*x['esort_5'] + 
                        dico_model_coefs['model_f5']['esort_4']*x['esort_4'] + 
                        dico_model_coefs['model_f5']['intercept'] <= pmax['f5'], name='pmax_f5')


pmax_ct_6 = m.addConstr(dico_model_coefs['model_f6']['larg']*larg + 
                        dico_model_coefs['model_f6']['esort_5']*x['esort_5'] + 
                        dico_model_coefs['model_f6']['esort_6']*x['esort_6'] + 
                        dico_model_coefs['model_f6']['intercept'] <= pmax['f6'], name='pmax_f6')


pmax_ct_7 = m.addConstr(dico_model_coefs['model_f7']['larg']*larg + 
                        dico_model_coefs['model_f7']['esort_7']*esort + 
                        dico_model_coefs['model_f7']['esort_6']*x['esort_6'] + 
                        dico_model_coefs['model_f7']['intercept'] <= pmax['f7'], name='pmax_f7')





In [106]:
#m.getObjective().addConstant(dico_coef_global['epentr_1']*ep_entree)

m.update()

In [107]:
m.write('laminage.lp')

In [108]:
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 14 rows, 6 columns and 24 nonzeros
Model fingerprint: 0xc533e0f9
Coefficient statistics:
  Matrix range     [3e-01, 4e+00]
  Objective range  [7e-02, 9e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 7 rows and 0 columns
Presolve time: 0.00s
Presolved: 7 rows, 6 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.5427645e+29   8.571787e+29   2.542764e-01      0s
       4    4.0611241e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.061124098e+01


In [109]:
all_vars = {v.varName: v.x for v in m.getVars()}

In [110]:
all_vars

{'esort_4': 4.444444462222222,
 'esort_6': 2.50000001,
 'esort_1': 10.53497946600823,
 'esort_2': 7.901234599506172,
 'esort_5': 3.3333333466666666,
 'esort_3': 5.9259259496296295}

In [114]:
dico_model_coefs['model_f1']['larg']*larg +dico_model_coefs['model_f1']['epentr_1']*ep_entree +dico_model_coefs['model_f1']['esort_1']*10.53497946600823 +dico_model_coefs['model_f1']['intercept']

8.799006703283169

In [116]:
7.33+8+7.38+7.31+7.32+6.35+3.22

46.910000000000004

In [115]:
data.loc[date_id]

pf1            7.324940
pf2            7.995752
pf3            7.371256
pf4            7.311965
pf5            7.320080
pf6            6.352902
pf7            3.228271
metal          4.000000
larg        1463.236900
esort_1       20.027542
esort_2       11.692580
esort_3        7.639256
esort_4        5.044345
esort_5        3.751749
esort_6        2.855336
esort_7        2.500000
epentr_1      34.000000
Name: 31/12/2024 16:55:44, dtype: float64